# MRE Quality ML Prediction

### MLFlow Pre-packaged Model Server AB Test Deployment on Kubernetes distrubuted compute with GPUs 
In this example powered by *ADEPT* methodogies, we will build two models with MLFlow and we will deploy them as an A/B test deployment. The reason this is powerful is because it allows you to deploy a new model next to the old one, distributing a percentage of traffic. These deployment strategies are quite simple using Seldon, and can be extended to shadow deployments, multi-armed-bandits, etc.

Tutorial Overview
This tutorial will follow closely break down in the following sections:

1. Train the MLFlow elastic net MRE Quality example

2. Deploy your trained model leveraging MLFlow model server

3. Test the deployed MLFlow model by sending requests

4. Deploy your second model as an A/B test

5. Visualise and monitor the performance of your models with automated self-service dashboards for decsion engineering!

In [1]:
### temp fix for PoC - fix deps, this takes a few minutes and wont be in production version

## pip uninstall itsdangerous werkzeug scikit-learn --yes
## pip install itsdangerous==2.0.1 werkzeug==2.0.3 scikit-learn==1.1.2 pandas

In [2]:
import os
import sys
from s3 import init_aws_creds
import warnings
import numpy as np
import pandas as pd
import mlflow.sklearn
import werkzeug as wkz
import mlflow, sklearn
import itsdangerous as itd

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

from seldon_core.seldon_client import SeldonClient

In [3]:
##  Reload the notebook kernel.

print('v' + itd.__version__)        ##  Output: v2.0.1
print('v' + wkz.__version__)        ##  Output: v2.0.3
print('v' + sklearn.__version__)    ##  Output: v1.1.2
print('v' + mlflow.__version__)     ##  Output: v1.14.1

v2.0.1
v2.0.3
v1.1.2
v1.14.1


In [4]:
 ## replace tracking URI from ADEPT/Aktiver GUI Pipeline Catalog page for MLFlow
MLFLOW_MODEL_TRACKING = 'http://54.91.117.197:32682/'

In [5]:
mlflow.set_tracking_uri(MLFLOW_MODEL_TRACKING)
mlflow.set_experiment("ADEPT_MRE_DEMO")

### Define MLflow project with MLproject file:
Next, project uses Conda for the environment and that it’s defined in the conda.yaml file.
Lastly, we can also see that the training will be performed by the train.py file, which receives two parameters ```alpha``` and ```l1_ratio```.

*Note: It is important to version lock the software, as in ```conda.yaml``` as a step towards reproducible data science.

In [6]:
!pygmentize -l yaml MLproject
!pygmentize conda.yaml

name: mre-quality

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"
name: mre-quality
channels:
  - defaults
dependencies:
  - python=3.9
  - scikit-learn=v1.1.2
  - pip:
    - mlflow=1.14.1


In [7]:
data = pd.read_csv("mre-quality.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [25]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the mre-quality csv file (make sure you're running this from the root of MLflow!)
mre_path = "mre-quality.csv"
data = pd.read_csv(mre_path)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

params = {"alpha": 0.5, "l1_ratio": 0.5}

mlflow.sklearn.autolog()
with mlflow.start_run():
    run = mlflow.active_run()
    print("MLFlow Run UUID: " + run._info.run_id)
    init_aws_creds(run._info.run_id)
    lr = ElasticNet(alpha=params["alpha"], l1_ratio=params["l1_ratio"], random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (params["alpha"], params["l1_ratio"]))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", params["alpha"])
    mlflow.log_param("l1_ratio", params["l1_ratio"])
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    
    
    # https://adeptdemo.s3.amazonaws.com/artifacts/1/c8962777bd2e4faab7697fabbf311ae8/artifacts/model/conda.yaml
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path='/model/',
        registered_model_name="adept_model-v"
    )
    
    #mlflow.sklearn.log_model(lr, f'''https://adeptdemo.s3.amazonaws.com/artifacts/1/{run._info.run_id}/artifacts/model''')
    #print("Model artifacts saved to: " + f'''https://adeptdemo.s3.amazonaws.com/artifacts/1/{run._info.run_id}/artifacts/model/''')
    #mlflow.log_artifacts(f'''https://adeptdemo.s3.amazonaws.com/artifacts/1/{run._info.run_id}/artifacts/model''')

MLFlow Run UUID: 99abed6b4e8346849fe78f0404c278e9
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689


Registered model 'adept_model-v' already exists. Creating a new version of this model...
2022/09/06 22:02:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: adept_model-v, version 6
Created version '6' of model 'adept_model-v'.
